# LSGYAN
- Least Squares Generative Adversarial Networks for GYAN
- https://arxiv.org/pdf/1611.04076.pdf

In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten, Activation
from keras.layers import BatchNormalization, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import keras.backend as K
from keras import regularizers
from keras.utils import generic_utils

import numpy as np
import matplotlib.pyplot as plt
import datetime, pathlib

In [ ]:
save_dir = "{0:%Y-%m-%d_%H%M%S}".format(datetime.datetime.now())
print(save_dir)

p = pathlib.Path(save_dir)
if p.exists() == False:
    p.mkdir()

In [ ]:
pixel_w = 128
pixel_h = 128
channel = 3

x_train = np.load('gyan_images_{}.npy'.format(pixel_w))

for i in range(x_train.shape[0]):
    print(i)
    plt.imshow(x_train[i,:,:,:].astype('uint8'))
    plt.show()

x_train = x_train.astype('float32')
x_train /= 255

In [ ]:
batch_size = 16
half_batch_size = int(batch_size / 2)
z_dim = 10

## Generator

In [ ]:
def generator_model():
    in_h = int(pixel_h / 4)
    in_w = int(pixel_w / 4)

    model = Sequential()
    model.add(Dense(256, input_shape=(z_dim,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Dense(in_h*in_w*128))
    model.add(Reshape((in_h, in_w, 128)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
        
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Conv2D(channel, (3, 3), padding="same", activation='sigmoid'))

    #model.summary()
    return model

## Discriminator
- Replace pooling layers with strided-convolutions
- Don't use BatchNormalization because I cannot generate normally.

In [ ]:
def discriminator_model():    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), strides=(2, 2), padding='same', input_shape=(pixel_h, pixel_w, channel)))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(64, (3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(128, (3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(256, (3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(LeakyReLU(0.2))
    
    model.add(Flatten())
    model.add(Dense(1))
    
    # Don't use activation at output layer
#    model.add(Activation('sigmoid'))

    #model.summary()
    return model

# Combine Generator and Discriminator

In [ ]:
def generator_and_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    #model.summary()
    return model

## Optimizer

In [ ]:
optimizer = Adam(0.0002, 0.5)

## Loss Function
- $a$ and $b$ are the labels for fake data
- $c$ denotes the value that G wants D to believe for fake data.

In [ ]:
# Parameters Selection
# See the paper 3,2,3 (9)
a, b, c = 0, 1, 1
# lambda
r = 0.9

# for Discriminator
def loss_D(y_true, y_pred):
    a_mask = K.cast(K.equal(y_true, a), K.floatx())
    b_mask = K.cast(K.equal(y_true, b), K.floatx())
    a_loss = K.sum((y_pred * a_mask - a) ** 2) / K.sum(a_mask)
    b_loss = K.sum((y_pred * b_mask - b) ** 2) / K.sum(b_mask)
    return 0.5 * (a_loss + b_loss)

# for Generator
def loss_G(y_true, y_pred):
    return 0.5 * (K.sum((y_pred - c) ** 2)) / batch_size + r * K.sum(K.abs(y_pred - y_true)) / batch_size

In [ ]:
discriminator = discriminator_model()
discriminator.compile(loss=loss_D, optimizer=optimizer, metrics=["accuracy"])

In [ ]:
generator = generator_model()
combined = generator_and_discriminator(generator, discriminator)
combined.compile(loss=loss_G, optimizer=optimizer, metrics=["accuracy"])

In [ ]:
def show_img(epoch, is_save=True):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r*c, z_dim))

    gen_imgs = generator.predict(noise)
    fig, axs = plt.subplots(r, c, figsize=(16, 16))
    cnt = 0
    for i in range(r):
        for j in range(c):
            img = np.squeeze(gen_imgs[cnt, :, :, :])
            axs[i, j].imshow(img)
            axs[i, j].axis('off')
            cnt += 1
    if is_save:
        fig.savefig(save_dir + "/lsgyan_%d.png" % epoch)
        plt.close()
    else:
        plt.show()

In [ ]:
epochs = 50000

d_loss_list = []
d_acc_list = []
g_loss_list = []
g_acc_list = []

for epoch in range(epochs):
    # Discriminator
    idx = np.random.randint(0, x_train.shape[0], half_batch_size)
    imgs = x_train[idx]
    # Create fake images
    noise = np.random.normal(0, 1, (half_batch_size, z_dim))
    gen_imgs = generator.predict(noise)
    # ombine Real + Fake images
    concat_imgs = np.concatenate([imgs, gen_imgs], axis=0)
    # Real label=1、Fake label=0
    concat_labels = np.concatenate([np.ones((half_batch_size, 1)), np.zeros((half_batch_size, 1))], axis=0)
    # train
    d_loss, d_acc = discriminator.train_on_batch(concat_imgs, concat_labels)
    d_loss_list.append(d_loss)
    d_acc_list.append(d_acc)

    # Generator
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    g_loss, g_acc = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    g_loss_list.append(g_loss)
    g_acc_list.append(g_acc)

    if epoch % 500 == 0:
        print("epoch:{} d_loss:{:2f} d_acc:{:2f} g_loss:{:2f} g_acc:{:2f}".format(epoch, d_loss, d_acc, g_loss, g_acc))
        #generator.save_weights(save_dir + "/generator_{}.hdf5".format(epoch))
        #discriminator.save_weights(save_dir + "/discriminator_{}.hdf5".format(epoch))
        show_img(epoch)

In [ ]:
plt.plot(d_loss_list, label='D Loss')
plt.plot(g_loss_list, label='G Loss')
plt.legend()
plt.show()

In [ ]:
show_img(0, False)

# Generate images while moving latent variables

In [ ]:
noise = np.random.normal(0, 1, (2, z_dim))
start = np.expand_dims(noise[0], axis=0)
end = np.expand_dims(noise[1], axis=0)
steps = 10
start_img = generator.predict(start)
end_img = generator.predict(end)

vectors = []
alpha_values = np.linspace(0, 1, steps)
for alpha in alpha_values:
    vector = start * (1 - alpha) + end * alpha
    vectors.append(vector)
vectors = np.array(vectors)

result_image = []
for i, vec in enumerate(vectors):
    gen_img = np.squeeze(generator.predict(vec), axis=0)
    gen_img *= 255
    interpolated_img = gen_img.astype('uint8')
    result_image.append(interpolated_img)

result_image = np.array(result_image)
r = 2
c = 5
fig, axs = plt.subplots(r, c, figsize=(16, r * 3))
cnt = 0
for i in range(r):
    for j in range(c):
        img = np.squeeze(result_image[cnt, :, :, :])
        axs[i, j].imshow(img)
        axs[i, j].axis('off')
        cnt += 1
plt.show()  